# Imports

In [1]:
import conv2visit_level as c2v
import numpy as np
import pandas as pd
import synth_metrics as s_m
import methods as met

/Users/nunomachado/anaconda3/envs/synth_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# conv2visit_level file tests

path to folder with OMOP csv files

In [2]:
folder = ''
folder = 'mimic_iv_OMOP_CDM/eda/'

- Conversion to visit level representation (each line is a single visit)
- A patient can have several visits (in several lines)
- function takes path to folder with OMOP csv's


In [3]:
df_visit = c2v.conv2visit_level_df(folder, remove_empty_visits = True)

In [4]:
df_visit.head(3)

,person_id,visit_occurrence_id,gender_concept_id,year_of_birth,age,visit_concept_id,visit,no_of_concepts,unique_concepts,no_of_unique_concepts
0,-8492299714241840941,3820211643995829887,8507,2043,70,38004207,[4201099],1,[4201099],1
1,8090044958540695372,6734319289435533605,8532,2079,64,38004207,"[40163721, 942384, 942384, 19068781, 40174979,...",38,"[138384, 314666, 319835, 443587, 942384, 96743...",34
2,-4873075614181207858,-5667932875078290562,8507,2075,65,38004207,"[40163667, 19072321, 967434, 19049684, 1904968...",21,"[317009, 317576, 320128, 432867, 967434, 11128...",19


- apply filtering (removing concepts that appear less than 'min_no_of_occ' times in all the visits)
- returns new df with filtered visits in new collumn
- column indicates the column to be filtered

In [5]:
df_visit2 = c2v.filter_visits(df_visit,
                              column = 'unique_concepts',
                              min_no_of_occ = 10)

counts-->Counter({4078442: 8500, 4145513: 6531, 4021323: 3326, 4064452: 2405, 4140598: 2302, 42536500: 2020, 4213288: 1144, 4141149: 1081, 46275280: 794, 4322380: 712, 40221385: 644, 0: 546, 4092038: 502, 19127213: 462, 40220357: 397, 4138456: 390, 4049832: 347, 40166274: 307, 43011850: 296, 19071700: 291, 1366754: 235, 40167213: 224, 19135374: 220, 44784779: 196, 35603226: 185, 40232756: 184, 1560751: 183, 938330: 183, 433753: 181, 19078557: 175, 4163872: 171, 40141424: 170, 40221350: 170, 36249735: 170, 19049684: 166, 35603227: 166, 35605482: 156, 1593611: 153, 4089488: 153, 36249737: 149, 19016865: 146, 1718752: 146, 43647680: 146, 35603636: 144, 36249739: 139, 19049686: 134, 1127433: 133, 40229093: 128, 43011884: 127, 4197894: 126, 19129446: 122, 1560525: 117, 40221387: 115, 40167196: 114, 432867: 112, 1113346: 106, 40221386: 103, 320128: 100, 1596977: 99, 19020131: 99, 36249736: 99, 46275400: 97, 21125875: 85, 19049024: 84, 939871: 84, 36249734: 82, 46276333: 81, 19020053: 78, 190

In [6]:
df_visit2.head(3)

,person_id,visit_occurrence_id,gender_concept_id,year_of_birth,age,visit_concept_id,visit,no_of_concepts,unique_concepts,no_of_unique_concepts,filtered_unique_concepts,no_of_concepts_in_filtered_unique_concepts
0,8090044958540695372,6734319289435533605,8532,2079,64,38004207,"[40163721, 942384, 942384, 19068781, 40174979,...",38,"[138384, 314666, 319835, 443587, 942384, 96743...",34,"[138384, 314666, 319835, 942384, 967434, 11274...",23
1,-4873075614181207858,-5667932875078290562,8507,2075,65,38004207,"[40163667, 19072321, 967434, 19049684, 1904968...",21,"[317009, 317576, 320128, 432867, 967434, 11128...",19,"[317009, 317576, 320128, 432867, 967434, 11274...",14
2,7918537411740862407,7730200099818586525,8532,2055,74,38004207,"[19035247, 40174888, 19133566, 40162494, 15607...",22,"[314666, 315286, 318800, 320128, 432845, 43286...",22,"[314666, 318800, 320128, 432867, 442588, 11274...",14


- conversion to txt format to be used in the fine tuning of the gpt model
- txt file is outputted to output_file_path
- visit_column selects the column with visits to be converted (ie: from the normal one, the one with only unique values and the filtered one)

In [7]:
df_visit3 = c2v.conv2txt4training(df_visit2,
                                  output_file_path = 'formatted_data_for_training_patient_visit.txt',
                                  visit_column = 'filtered_unique_concepts')

In [8]:
df_visit3.head(3)

,person_id,visit_occurrence_id,gender_concept_id,year_of_birth,age,visit_concept_id,visit,no_of_concepts,unique_concepts,no_of_unique_concepts,filtered_unique_concepts,no_of_concepts_in_filtered_unique_concepts,formatted
0,8090044958540695372,6734319289435533605,8532,2079,64,38004207,"[40163721, 942384, 942384, 19068781, 40174979,...",38,"[138384, 314666, 319835, 443587, 942384, 96743...",34,"[138384, 314666, 319835, 942384, 967434, 11274...",23,<GID> 8532 <YOB> 2079 <AGE> 64 <VCID> 38004207...
1,-4873075614181207858,-5667932875078290562,8507,2075,65,38004207,"[40163667, 19072321, 967434, 19049684, 1904968...",21,"[317009, 317576, 320128, 432867, 967434, 11128...",19,"[317009, 317576, 320128, 432867, 967434, 11274...",14,<GID> 8507 <YOB> 2075 <AGE> 65 <VCID> 38004207...
2,7918537411740862407,7730200099818586525,8532,2055,74,38004207,"[19035247, 40174888, 19133566, 40162494, 15607...",22,"[314666, 315286, 318800, 320128, 432845, 43286...",22,"[314666, 318800, 320128, 432867, 442588, 11274...",14,<GID> 8532 <YOB> 2055 <AGE> 74 <VCID> 38004207...


- conversion to simple format with concept descriptions
- column to be used
- output

In [9]:
df_visit4 = c2v.simple_format_descriptions(df_visit2,
                                       visit_column = 'filtered_unique_concepts',
                                       output_file_path = 'trial_desc.txt')

/Users/nunomachado/Documents/HL/synth_data/conv2visit_level.py:359: DtypeWarning: Columns (6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_concept = pd.read_csv(concept_data_path, on_bad_lines='skip', sep='\t')


In [10]:
df_visit4.head(4)['formatted'].tolist()

['Female, born in 2079, aged 64 when admitted in a Ambulatory Clinic / Center, had the following events: [Acquired hypothyroidism, Old myocardial infarction, Congestive heart failure, torsemide 20 MG Oral Tablet, simethicone 80 MG Chewable Tablet, acetaminophen 325 MG Oral Tablet, 50 ML glucose 500 MG/ML Prefilled Syringe, insulin, regular, human, Diabetes mellitus without complication, glucagon (rDNA) 1 MG Injection [GlucaGen], lorazepam 0.5 MG Oral Tablet, carvedilol 12.5 MG Oral Tablet, aspirin 325 MG Oral Tablet, potassium chloride 10 MEQ Extended Release Oral Tablet, atorvastatin 80 MG Oral Tablet [Lipitor], insulin glargine 100 UNT/ML Injectable Solution [Lantus], spironolactone 25 MG Oral Tablet, ezetimibe 10 MG Oral Tablet [Zetia], sodium chloride Prefilled Syringe, trazodone hydrochloride 50 MG Oral Tablet, 1000 ML sodium chloride 4.5 MG/ML Injection, Benign hypertensive renal disease with renal failure, Chronic kidney disease]|',
 'Male, born in 2075, aged 65 when admitted in

In [11]:
df_visit4.head(3)

,person_id,visit_occurrence_id,gender_concept_id,year_of_birth,age,visit_concept_id,visit,no_of_concepts,unique_concepts,no_of_unique_concepts,filtered_unique_concepts,no_of_concepts_in_filtered_unique_concepts,formatted,No. of unknowns
0,8090044958540695372,6734319289435533605,8532,2079,64,38004207,"[40163721, 942384, 942384, 19068781, 40174979,...",38,"[138384, 314666, 319835, 443587, 942384, 96743...",34,"[138384, 314666, 319835, 942384, 967434, 11274...",23,"Female, born in 2079, aged 64 when admitted in...",0
1,-4873075614181207858,-5667932875078290562,8507,2075,65,38004207,"[40163667, 19072321, 967434, 19049684, 1904968...",21,"[317009, 317576, 320128, 432867, 967434, 11128...",19,"[317009, 317576, 320128, 432867, 967434, 11274...",14,"Male, born in 2075, aged 65 when admitted in a...",0
2,7918537411740862407,7730200099818586525,8532,2055,74,38004207,"[19035247, 40174888, 19133566, 40162494, 15607...",22,"[314666, 315286, 318800, 320128, 432845, 43286...",22,"[314666, 318800, 320128, 432867, 442588, 11274...",14,"Female, born in 2055, aged 74 when admitted in...",0


## Patient level rep

- conversion to patient level representation (each row is a single patient)
- visits by a patient are separated by [W0] values indicating the time between them.
- ie: W1 -> one week; M7 -> 7 months
- uniques and filterings by minimum no. of occurrences is already applied and is in the df

In [12]:
df_pat = c2v.conv2pat_level_df(folder_path = folder,
                               remove_empty_visits = True,
                               filter_by = 10)

In [13]:
df_pat.head(3)

,person_id,gender_concept_id,year_of_birth,visits,no_of_visits,visit_types,no_of_concepts,visit_with_unique_codes,no_of_concepts_in_visit_with_unique_codes,visit_filtered,no_of_filtered_concepts,filtered_visit_types,visit_unique_filtered,no_of_filtered_unique_concepts,no_filtered_visits
0,3589912774911670296,8507,2095,"[VS, 40166274, 40166274, 19078557, 19078557, 1...",1,[262],360,"[VS, 0, 717166, 961050, 963359, 1113346, 11274...",101,"[VS, 40166274, 40166274, 19078557, 19078557, 1...",336,[262],"[VS, 0, 717166, 961050, 963359, 1113346, 11274...",85,1
1,-3210373572193940939,8507,2079,"[VS, 19078557, 40163667, 40163667, 19065567, 1...",1,[8883],234,"[VS, 0, 717166, 961050, 963359, 1113346, 11274...",79,"[VS, 19078557, 40163667, 40163667, 19065567, 1...",223,[8883],"[VS, 0, 717166, 961050, 963359, 1113346, 11274...",70,1
2,-775517641933593374,8507,2149,"[VS, 40221385, 19078557, 19078557, 19078557, 1...",3,"[262, 8883, 8870]",667,"[VS, 0, 902807, 938330, 948890, 953102, 130825...",171,"[VS, 40221385, 19078557, 19078557, 19078557, 9...",630,"[262, 8883, 8870]","[VS, 0, 938330, 948890, 953102, 1308251, 13099...",137,3


# Simple baseline

- just calculates the distribution of each event (procedure_occ, condition_occ and drug_exposure) and generates visits based on this
- like in the generation of data with the LLM where the prompting pre determines the age at visit start, year of birth and visit type according to the population. the same will be done here

In [14]:
df_visit_synth = met.baseline_sample(df_visit, 20)

In [15]:
df_visit_synth.head(3)

,gender_concept_id,year_of_birth,age,visit_concept_id,visit,no_of_concepts
0,8507,2062,80,581385,"[43647680, 314666, 4140598, 4078442, 4064452, ...",1150
1,8507,2094,68,262,"[0, 1593063, 46275280, 19009384, 4021323, 4301...",185
2,8532,2086,74,8870,"[4138456, 4078442, 4145513, 4021323, 40167196,...",96


In [16]:
df_visit_synth2 = c2v.conv2txt4training(df_visit_synth,
                                  output_file_path = 'baseline.txt',
                                  visit_column = 'visit')

# Testing Fine tuning of LLM and  generation of synth data with it

on google collab

In [9]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.5 MB/s eta 0:00:00


does fine tuning with gpt 2 model with data from specified file generated with tht c2v.conv2txt4training function
fine tuned model is saved to folder model_gpt2_finetuned and also outputted with the tokenizer from gpt2

In [5]:
model, tokenizer = met.llm_finetuning('formatted_data_for_training_patient_visit.txt')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
500,1.417800
1000,0.799100


model and tokenizer save to model_gpt2_finetuned folder


generates N synthetic data records in txt format, using the fine tuned model.
first argument is the df_visits generated with c2v NOT the visit_occ df in OMOP

In [6]:
met.generate_synth_data_ids_format(df_visit, model, tokenizer, N = 50)

Synthetic records saved to synthetic_records_p80_prompt_attribute.txt


# Synth_Metrics file tests

- Takes real and synthetic data txt files.
- txt's from the format used to fine tune the gpt2 model
- specify method used in the generation and folder to save plots and metric values
- saves the prevalence plots for each domain
- print to txt the log cluster and kl div values

In [19]:
s_m.all_stats_ids_format(real_data_path = 'formatted_data_for_training_patient_visit.txt',
                         synthetic_data_path = 'baseline.txt' ,
                         method_name = 'gpt2_person',
                         output_folder_path = 'Plots/Baseline/')

/Users/nunomachado/Documents/HL/dummy_dataset/organizado/conv2visit_level.py:359: DtypeWarning: Columns (6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_concept = pd.read_csv(concept_data_path, on_bad_lines='skip', sep='\t')
/Users/nunomachado/Documents/HL/dummy_dataset/organizado/conv2visit_level.py:372: DtypeWarning: Columns (6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_concept = pd.read_csv(concept_data_path, on_bad_lines='skip', sep='\t')


Procedure plot saved to: Plots/Baseline/gpt2_person_prevalance_Procedure.png

Drug plot saved to: Plots/Baseline/gpt2_person_prevalance_Drug.png

Condition plot saved to: Plots/Baseline/gpt2_person_prevalance_Condition.png

Visit plot saved to: Plots/Baseline/gpt2_person_prevalance_Visit.png



/Users/nunomachado/anaconda3/envs/synth_env/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/nunomachado/Documents/HL/dummy_dataset/organizado/synth_metrics.py:162: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_feature_vectors['cluster'] = clusters


log_cluster_value-->-1.8772881000501602
kl-->0.053243133423007356


# Reconversion tests (not working)

- reconversion to the codes
- (not working)

In [ ]:
data = c2v.reconvert_from_descriptions(file_path = 'trial_desc.txt')

/Users/nunomachado/Documents/HL/dummy_dataset/organizado/conv2visit_level.py:361: DtypeWarning: Columns (6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_concept = pd.read_csv(concept_data_path, on_bad_lines='skip', sep='\t')


{'gender': 8532, 'year_of_birth': '2079', 'age': '64', 'visit_type': 38004207, 'events': [138384, 314666, 319835, 942384, 967434, 1127433, 1560751, 'insulin', 'regular', 'human', 4008576, 19016865, 19019112, 19022750, 19046742, 19049684, 36943892, 36963911, 19079658, 19098474, 40141424, 40163473, 40221369, 44784439, 46271022]}
{'gender': 8507, 'year_of_birth': '2075', 'age': '65', 'visit_type': 38004207, 'events': [40410639, 317576, 320128, 432867, 967434, 1127433, 19049684, 19072321, 40141424, 40163667, 40167213, 40167218, 40221369, 40231925]}
{'gender': 8532, 'year_of_birth': '2055', 'age': '74', 'visit_type': 38004207, 'events': [314666, 318800, 320128, 432867, 442588, 1127433, 1560525, 1560751, 4008576, 19016865, 19035247, 19127213, 40164929, 40174888]}
{'gender': 8507, 'year_of_birth': '2075', 'age': '68', 'visit_type': 38004207, 'events': [967434, 1127433, 19035247, 19040187, 19127213, 19133574, 40163473, 40232756, '0.5 ML influenza A virus vaccine', 'A-Victoria-361-2011 (H3N2)-l

In [ ]:
data

[{'gender': 8532,
  'year_of_birth': '2079',
  'age': '64',
  'visit_type': 38004207,
  'events': [138384,
   314666,
   319835,
   942384,
   967434,
   1127433,
   1560751,
   'insulin',
   'regular',
   'human',
   4008576,
   19016865,
   19019112,
   19022750,
   19046742,
   19049684,
   36943892,
   36963911,
   19079658,
   19098474,
   40141424,
   40163473,
   40221369,
   44784439,
   46271022]},
 {'gender': 8507,
  'year_of_birth': '2075',
  'age': '65',
  'visit_type': 38004207,
  'events': [40410639,
   317576,
   320128,
   432867,
   967434,
   1127433,
   19049684,
   19072321,
   40141424,
   40163667,
   40167213,
   40167218,
   40221369,
   40231925]},
 {'gender': 8532,
  'year_of_birth': '2055',
  'age': '74',
  'visit_type': 38004207,
  'events': [314666,
   318800,
   320128,
   432867,
   442588,
   1127433,
   1560525,
   1560751,
   4008576,
   19016865,
   19035247,
   19127213,
   40164929,
   40174888]},
 {'gender': 8507,
  'year_of_birth': '2075',
  'age'

In [ ]:
def invert_dictionary(dictionary):
    return {v: k for k, v in dictionary.items()}

In [ ]:
events_dict = c2v.get_concept_dict()

/Users/nunomachado/Documents/HL/dummy_dataset/organizado/conv2visit_level.py:361: DtypeWarning: Columns (6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_concept = pd.read_csv(concept_data_path, on_bad_lines='skip', sep='\t')


In [ ]:
inverted_events_dict = invert_dictionary(events_dict)

In [ ]:
inverted_events_dict['porcine 5000 UNT/ML Injectable Solution']

KeyError: 'porcine 5000 UNT/ML Injectable Solution'